In [1]:
import torch
import sys
sys.path.append("../")
from shared_utils.load import get_tokenizer, configs_from_yaml
from shared_utils.generate import generate_text
from early_exit.util import get_model, load_model
from early_exit.patching import replace_attention_layers, set_transformer_early_exit_mode
from early_exit_teacher.visualization import create_html_visualization, visualize_tokens_by_exit_layer, safe_decode_tokens

model_path = "../models/early_exit_20250811_layers_5_big"
base_model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
config_path = "../config_deepseek.yaml"
device = "cuda"

In [2]:
tokenizer = get_tokenizer(base_model)
config = configs_from_yaml(config_path, tokenizer.eos_token_id)

base_model = get_model(base_model, config['model'], device)
model = replace_attention_layers(base_model, config['lora'], device)
model = load_model(model, model_path)

print(f"Model loaded w exitable layers: {model.exitable_layer_idxs}")

address this hack!
g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

trainable params: 2,179,072 || all params: 1,802,890,757 || trainable%: 0.1209
Model loaded w exitable layers: tensor([ 5., 10., 15., 20., 25., inf])


In [3]:
set_transformer_early_exit_mode(model, 'free_generate')

In [4]:
prompt = "Explain the concept of recursion in programming."
# prompt = "What are the main causes of climate change?"
system_prompt = "You are a helpful programming tutor."
prefiller = ""

In [5]:
config['generation']['max_new_tokens'] = 400
with torch.no_grad():
    try:
        free_generate_response, exit_info = generate_text(
            model=model,
            prompt=prompt,
            system_prompt=system_prompt,
            prefiller=prefiller,
            tokenizer=tokenizer,
            generation_config=config['generation'],
            device=device
        )
        
        print(f"Free Generate Response: {free_generate_response}")
        # print(f"Exit info: {exit_info}")
        
    except Exception as e:
        print(f"Free generate mode failed: {e}")

transform_conversations currently only for Deepseek models!
full_tokenize currently only for Deepseek models!
prompt tokens shape: torch.Size([1, 20])
Free Generate Response: <｜begin▁of▁sentence｜><｜Assistant｜> You are a helpful programming tutor.
<｜User｜> Explain the concept of recursion in programming.
<｜Assistant｜> 
Okay, so I need to explain how recursion works in programming. Hmm, I remember hearing about it somewhere in my computer science class. Let me try to break it down.

First, recursion is when a function calls itself. That means the function has a base value/value that you stop/stop掉, and then it calls itself again with a smaller or other agreed-upon version of borrowed shares. So, it's like a loop where each successive iteration brings closer to the solution.

Wait, how does that work exactly? Let's say you have a function that calculates factorial. Factorial of n is n times the factorial of (n-1). So, if I call factorial(3，则 it's equal to 3 * itself(2). And itself(2) is e

In [6]:
from early_exit.util import module_name_is_layer_base
early_exit_layer_idxs = []
for name, module in model.named_modules():
    if module_name_is_layer_base(name):
        layer_idx = int(name.split('.')[-1])
        early_exit_layer_idxs.append(layer_idx)

early_exit_layer_idxs = torch.tensor(early_exit_layer_idxs, dtype = torch.int32)  # Add inf for final layer

In [7]:
# tokens = [tokenizer.decode([token]) for token in exit_info[0][0, 20:]]
tokens = safe_decode_tokens(tokenizer, exit_info[0][0, 20:])
layers = [27 if item == torch.inf or item == -1 else int(item) for item in exit_info[1][0]]
early_exit_layers = early_exit_layer_idxs.tolist()  # Convert tensor to list if needed
# Display the visualization
display(visualize_tokens_by_exit_layer(tokens, layers, early_exit_layers, 
                                     title="Committed Early Exit Token Generation"))